In [1]:
#importing modules we require 
import cv2
import numpy as np    
import math   
import matplotlib.pyplot as plt   
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize  

Using TensorFlow backend.


In [2]:
def Frame_by_Frame(videoFile):
    count=0
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            filename ="frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    print ("Done!")

In [3]:
videoFile = "Traffic1.mp4"
#Frame_by_Frame(videoFile)

In [4]:
    n=cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    list_of_lables=[]
    with open("coco.names", "r") as f:
        list_of_labels=[line.strip() for line in f.readlines()]
    layers = n.getLayerNames()
    out_layers = [layers[i[0] - 1] for i in n.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(list_of_labels), 3))

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:214: error: (-212:Parsing error) Failed to parse NetParameter file: yolov3.weights in function 'cv::dnn::dnn4_v20191202::readNetFromDarknet'


In [ ]:
def Obj_detect(image_name):
    cur_img= cv2.imread(image_name)
    height, width, channels = cur_img.shape
    blob = cv2.dnn.blobFromImage(cur_img, 0.00392, (416,416), (0, 0, 0), True, crop=False)
    n.setInput(blob)
    outs = n.forward(out_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    count=0
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(list_of_labels[class_ids[i]])
            if(label in ['car','bus','truck','motorbike','bicycle']):
                #print(label,count,sep=" : ")
                count+=1
            color = colors[i]
            cv2.rectangle(cur_img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(cur_img, label, (x, y + 30), font, 3, color, 3)
    return count
    #cv2.imshow("Image", cur_img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

In [ ]:
total_chaos=0
no_of_frames=0
for i in range(0,127):
    imag_n="frame"+str(i)+".jpg"
    s=Obj_detect(imag_n)
    if(s!=0):
        total_chaos+=s
        no_of_frames+=1
    
    #Obj_detect("frame47.jpg")    

In [ ]:
print("Total Chaos : ",total_chaos)
print("Useful no of frames : ",no_of_frames)

In [ ]:
avg_chaos=(total_chaos/no_of_frames)
print("Average Chaos : ",avg_chaos)